<a href="https://colab.research.google.com/github/dj-1087/BigContest-COIN/blob/master/EndGame_Tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
pip install -U keras-tuner

     |████████████████████████████████| 61kB 1.9MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=f2e07a0558669d2ce0688ea088cd2394b10e1e77321b119f08c7bb852a5283ae
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=a59e1dadcb7aec8d5824fe8006a6d8f68c12a10599f16a7930ff8319dc9b4f19
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, LSTM, TimeDistributed
from tensorflow.keras.layers import Input, Dropout
from tensorflow.keras.optimizers import Adam

# returns train, inference_encoder and inference_decoder models
def define_models(n_input, n_output, n_units , hp):
	# define training encoder
	encoder_inputs = Input(shape=(-1,20, n_input))
	encoder = LSTM(n_units, return_state=True,activation='relu')
	encoder_outputs, state_h, state_c = encoder(encoder_inputs)
	encoder_states = [state_h, state_c]
    # define training encoder2
	encoder_inputs2 = Input(shape=(-1, 20, n_input))
	encoder2 = LSTM(n_units, return_state=True, activation='relu')
	encoder_outputs2, state_h2, state_c2 = encoder2(encoder_inputs2)
	encoder_states2 = [state_h2, state_c2]

    lstm_inputs = Input(shape=(-1, 10, n_input))
    decoder_inputs = LSTM(n_units, activation='relu')
    decoder_inputs,  _, _  = decoder_inputs(lstm_inputs)

	# define training decoder
	decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True)
	decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states, activation='relu')
    decoder_timedistribute = TimeDistributed(Dense(hp.Int('units', min_value = 4, max_value = 64, step = 4), activation='relu'))
    decoder_outputs = decoder_timedistribute(decoder_outputs)

 	# define training decoder2
	decoder_lstm2 = LSTM(n_input, return_sequences=True, return_state=True)
	decoder_outputs2, _, _ = decoder_lstm2(decoder_outputs, initial_state=encoder_states2, activation='relu')
    decoder_timedistribute2 = TimeDistributed(Dense(1, activation='sigmoid'))
	output = decoder_timedistribute2(decoder_outputs2)
 
	model = Model(inputs=[encoder_inputs, encoder_inputs2, decoder_inputs], outputs=decoder_outputs2)
	decoder_outputs2 = decoder_dense(decoder_outputs2)
	# model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
	# define inference encoder
	encoder_model = Model(encoder_inputs, encoder_states)
	# define inference decoder
	decoder_state_input_h = Input(shape=(n_units,))
	decoder_state_input_c = Input(shape=(n_units,))
	decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
	decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
	decoder_states = [state_h, state_c]
	decoder_outputs = decoder_dense(decoder_outputs)
	decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
 
   
    model.compile(optimizer=Adam(hp.Choice('learning_rate', values=[1e-3,1e-4,1e-5])),
                  loss = 'binary_crossentropy',
                  metrics=['binary_accuracy'])
	# return all models
	return model, encoder_model, decoder_model


IndentationError: ignored

In [ ]:
# define model
train, infenc, infdec = define_models(n_features, n_features, 128)
train.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
train.fit([X1, X2], y, epochs=1)

In [ ]:
# evaluate LSTM
total, correct = 100, 0
for _ in range(total):
	X1, X2, y = get_dataset(n_steps_in, n_steps_out, n_features, 1)
	target = predict_sequence(infenc, infdec, X1, n_steps_out, n_features)
	if array_equal(one_hot_decode(y[0]), one_hot_decode(target)):
		correct += 1
print('Accuracy: %.2f%%' % (float(correct)/float(total)*100.0))

In [ ]:
# returns train, inference_encoder and inference_decoder models
def define_models(n_input, n2_input, n_output, n_units):
	# define training encoder
	encoder_inputs = Input(shape=(None, n_input))
	encoder = LSTM(n_units, return_state=True)
	encoder_outputs, state_h, state_c = encoder(encoder_inputs)
	encoder_states = [state_h, state_c]
    # define training encoder2
	encoder_inputs2 = Input(shape=(None, n2_input))
	encoder2 = LSTM(n_units, return_state=True)
	encoder_outputs2, state_h2, state_c2 = encoder(encoder_inputs2)
	encoder_states2 = [state_h2, state_c2]
	# define training decoder
	decoder_inputs = Input(shape=(None, n_output))
	decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True)
	decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
	decoder_dense = Dense(n_output, activation='softmax')
	decoder_outputs = decoder_dense(decoder_outputs)
    # define training decoder2
	decoder_inputs2 = Input(shape=(None, decoder_outputs))
	decoder_lstm2 = LSTM(n_units, return_sequences=True, return_state=True)
	decoder_outputs2, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states2)
	decoder_dense2 = Dense(n_output, activation='softmax')
	decoder_outputs2 = decoder_dense(decoder_outputs2)
	model = Model([encoder_inputs, decoder_inputs], decoder_outputs2)
	# define inference encoder
	encoder_model = Model(encoder_inputs, encoder_states)
	# define inference decoder
	decoder_state_input_h = Input(shape=(n_units,))
	decoder_state_input_c = Input(shape=(n_units,))
	decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
	decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
	decoder_states = [state_h, state_c]
	decoder_outputs = decoder_dense(decoder_outputs)
	decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
	# return all models
	return model, encoder_model, decoder_model

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.layers import Input, Dropout
from tensorflow.keras.optimizers import Adam

In [ ]:
def build_encoder(n_input, hp):
    # define training encoder
	encoder_inputs = Input(shape=(None, n_input))
	encoder = LSTM(hp.Int('encoder_units', min_value=4, max_value=128, step=4), return_state=True, activation='relu')
	encoder_outputs, state_h, state_c = encoder(encoder_inputs)
	encoder_states = [state_h, state_c]
    return encoder_outputs, encoder_states

In [ ]:
def build_dencoder(decode_input, encoder_states, hp):
 	decoder_inputs = Input(shape=(None, decode_input))
	decoder_lstm = LSTM(hp.Int('encoder_units', min_value=4, max_value=128, step=4), return_sequences=True, return_state=True)
	decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
	decoder_dense = Dense(decode_input, activation='softmax')
	decoder_outputs = TimeDistributed(decoder_dense(decoder_outputs))
    return decoder_outputs

In [ ]:
# returns train, inference_encoder and inference_decoder models
def define_models(encode_input, encode2_input, decode_input, hp):
	# define training encoder
	encoder_outputs, encoder_states = build_encoder(encode_input, hp)
    # define training encoder2
	encoder_outputs2, encoder_states2 =  build_encoder(encode2_input, hp)
	# define training decoder
	decoder_outputs = build_dencoder(decode_input, encoder_states, hp)
    # define training decoder2
	decoder_outputs2 = build_dencoder(decode_outputs, encoder_states2, hp)
	model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
	# define inference encoder
	encoder_model = Model(encoder_inputs, encoder_states)
	# define inference decoder
	decoder_state_input_h = Input(shape=(n_units,))
	decoder_state_input_c = Input(shape=(n_units,))
	decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
	decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
	decoder_states = [state_h, state_c]
	decoder_outputs = decoder_dense(decoder_outputs)
	decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
	# return all models
	return model, encoder_model, decoder_model

In [ ]:

def build_model(hp):
    inputs = Input(shape=(28,28,1))
    x = inputs

    for i in range(hp.Int('n_layers',1,3)):
        #필터 개수 탐색
        x = Conv2D(filters=hp.Int('filters_'+str(i), 4, 64, step=8, default=16),
                   kernel_size = (3,3), activation='relu', padding='same')(x)
    x = MaxPooling2D(strides=2)(x)

    #GAP? GMP?
    if hp.Choice('global_pooling',['max','avg']) == 'avg':
        x = GlobalAveragePooling2D()(x)
    else:
        x = GlobalMaxPooling2D()(x)
    
    #은닉층의 개수 탐색
    x = Dense(units=hp.Int('units', min_value = 16, max_value = 128, step = 16),
              activation = 'relu')(x)
    
    #드롭아웃률 탐색
    x = Dropout(hp.Choice('dropout_rate', values = [0.2, 0.3, 0.5]))(x)
    x = Dense(10, activation="softmax")(x)

    model = Model(inputs=inputs, outputs=x)
    model.compile(optimizer=Adam(hp.Choice('learning_rate', values=[1e-3,1e-4,1e-5])),
                  loss = 'categorical_crossentropy',
                  metrics=['acc'])
    return model